In [1]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages: Annotated[list, add_messages]

In [2]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [3]:
from langchain_core.tools import tool
import requests

@tool
def get_date_and_time() -> dict:
    """
    Call tool to fetch the current date and time from an API.
    """
    try:
        response = requests.get("https://timeapi.io/api/Time/current/zone?timeZone=Europe/Brussels")
        response.raise_for_status()
        data = response.json()
        return {"date_time": data["dateTime"], "timezone": data["timeZone"]}
    except requests.RequestException as e:
        return {"error": str(e)}

In [4]:
GROQ_API_KEY="#"

In [5]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="deepseek-r1-distill-llama-70b")

In [6]:
llm_with_tools = llm.bind_tools([get_date_and_time])


In [9]:
from langgraph.graph import StateGraph
from langchain_core.runnables import RunnableConfig

def chatbot(state: State, config: RunnableConfig):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}



In [11]:
# initiate the graph
graph_builder = StateGraph(State)
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_edge("chatbot", "chatbot")

# ✅ Set entry point
graph_builder.set_entry_point("chatbot")

# ✅ Now compile
graph = graph_builder.compile(checkpointer=memory)
